In [ ]:
# @title ## 2.2 Batch Preprocessing of Ligand Files 💊 (Required)
# @markdown a. Please **click the Run button** ▶️ on the left and follow the prompts.
# @markdown
# @markdown b. After **uploading your files** 📤 (SDF, XML, or PDB format), the script will automatically process them 🔄.
# @markdown
# @markdown c. Once processing is completed ✅, the script will package the processed files 📦 and provide a download link 📥.

import os
import sys
import subprocess
import shutil
from google.colab import files

def setup_openbabel():
    """Set up OpenBabel environment"""
    print("🔧 Configuring OpenBabel environment...")

    try:
        !conda create -n openbabel_env python=3.7 -y
        !conda install -n openbabel_env -c conda-forge openbabel -y

        conda_prefix = subprocess.check_output(['conda', 'info', '--json'], text=True)
        import json
        conda_info = json.loads(conda_prefix)
        env_path = os.path.join(conda_info['conda_prefix'], 'envs', 'openbabel_env')

        # Update Python path
        site_packages = os.path.join(env_path, 'lib', 'python3.7', 'site-packages')
        if site_packages not in sys.path:
            sys.path.append(site_packages)

        # Update LD_LIBRARY_PATH
        lib_path = os.path.join(env_path, 'lib')
        os.environ['LD_LIBRARY_PATH'] = f"{lib_path}:{os.environ.get('LD_LIBRARY_PATH', '')}"

        import openbabel
        from openbabel import pybel
        print("✅ OpenBabel configuration successful!")
        return True

    except Exception as e:
        print(f"⚠️ OpenBabel configuration failed, trying alternative approach...")
        try:
            !apt-get update
            !apt-get install -y libopenbabel-dev python3-openbabel

            import openbabel
            from openbabel import pybel
            print("✅ OpenBabel installed successfully via system package manager!")
            return True

        except Exception as e2:
            print(f"❌ OpenBabel installation failed: {e2}")
            return False

print("🚀 Initializing environment...")
if not setup_openbabel():
    print("❌ Error: Unable to set up OpenBabel environment. Please check error messages and retry.")
    raise SystemExit(1)

print("📚 Importing required modules...")
from openbabel import pybel

def convert_to_pdb(input_file):
    """Convert SDF or XML format molecular structure files to PDB format"""
    try:
        mol = next(pybel.readfile(os.path.splitext(input_file)[1][1:], input_file))
        pdb_file = os.path.splitext(input_file)[0] + ".pdb"
        mol.write("pdb", pdb_file, overwrite=True)
        return pdb_file
    except Exception as e:
        print(f"❌ Error: Failed to convert file {input_file} to PDB format: {e}")
        return None

def run_mgltools_command(command, args, error_message):
    """Execute MGLTools command and handle errors"""
    try:
        result = subprocess.run(
            [mgltools_path, command] + args,
            capture_output=True,
            text=True,
            env=os.environ
        )
        if result.returncode != 0:
            print(f"⚠️ Warning: {error_message}. Error message: {result.stderr}")
            return False
        return True
    except Exception as e:
        print(f"⚠️ Warning: {error_message}. Exception: {e}")
        return False

def process_ligand_files():
    """Main function for processing ligand files"""
    print("📤 Please upload molecular structure files (SDF, XML, or PDB format)...")
    uploaded = files.upload()

    if not uploaded:
        print("❌ Error: No files uploaded. Please upload molecular structure files.")
        return

    # Create output directory
    output_directory = '/content/temp_ligands/'
    if not os.path.exists(output_directory):
        os.makedirs(output_directory)

    processed_files = []  # Track successfully processed files

    for filename in uploaded.keys():
        input_file = os.path.join('/content', filename)
        file_extension = os.path.splitext(input_file)[1].lower()

        # File format handling
        if file_extension in ['.sdf', '.xml']:
            print(f"\n🔄 Processing {file_extension.upper()} file: {filename}")
            pdb_file = convert_to_pdb(input_file)
            if pdb_file is None:
                continue
            print(f"✅ Conversion successful: {input_file} -> {pdb_file}")
            ligand_path = pdb_file
        elif file_extension == '.pdb':
            print(f"\n🔄 Processing PDB file: {filename}")
            # Create a working copy of the PDB file
            work_pdb = os.path.join(output_directory, filename)
            try:
                shutil.copy2(input_file, work_pdb)
                ligand_path = work_pdb
                print(f"✅ Copied PDB file to working directory: {work_pdb}")
            except Exception as e:
                print(f"❌ Error: Failed to copy PDB file: {e}")
                continue
        else:
            print(f"❌ Error: Unsupported file format {file_extension}. Please upload SDF, XML, or PDB format files.")
            continue

        # Verify file exists and is accessible
        if not os.path.exists(ligand_path):
            print(f"❌ Error: File not found {ligand_path}")
            continue

        try:
            with open(ligand_path, 'r') as f:
                
                pass
        except Exception as e:
            print(f"❌ Error: Cannot read file {ligand_path}: {e}")
            continue

        output_ligand_path = os.path.join(output_directory,
                                         os.path.basename(ligand_path).replace('.pdb', '.pdbqt'))

        # 1. Add hydrogens and charges, output as PDBQT format
        print(f"🔄 Processing {ligand_path}...")
        if not run_mgltools_command(
            prepare_ligand4_path,
            ["-l", ligand_path, "-o", output_ligand_path],
            f"Error processing file {ligand_path}"
        ):
            continue

        print(f"✅ Added hydrogens and charges, output file: {output_ligand_path}")

        if not os.path.exists(output_ligand_path):
            print(f"❌ Error: Output file {output_ligand_path} does not exist.")
            continue

        # 2. Automatically detect root
        if run_mgltools_command(
            prepare_ligand4_path,
            ["-l", output_ligand_path, "-A", "detect_root"],
            f"Error detecting root for file {output_ligand_path}"
        ):
            print(f"✅ Root detection completed for {ligand_path}")

        # 3. Select rotatable bonds
        if run_mgltools_command(
            prepare_ligand4_path,
            ["-l", output_ligand_path, "-A", "choose_torsions"],
            f"Error selecting rotatable bonds for file {output_ligand_path}"
        ):
            print(f"✅ Rotatable bond selection completed for {ligand_path}")

        processed_files.append(output_ligand_path)

    if processed_files:
        print(f"\n🎉 Successfully processed files:")
        for file in processed_files:
            print(f"- {file}")

        print("\n📦 Packaging processed files...")
        try:
            zip_name = "processed_ligands"
            shutil.make_archive(zip_name, 'zip', output_directory)
            print("📥 Providing download link...")
            files.download(f'{zip_name}.zip')
            print("🎉 All ligand files processed successfully.")
        except Exception as e:
            print(f"❌ Error: Failed to package files: {e}")
    else:
        print("⚠️ Warning: No files were successfully processed.")

    try:
        shutil.rmtree(output_directory)
    except Exception as e:
        print(f"⚠️ Warning: Error cleaning up temporary files: {e}")

if not all(var in globals() for var in ['mgltools_path', 'prepare_ligand4_path']):
    print("❌ Error: MGLTools environment variables not properly set. Please ensure you've run the environment configuration script first.")
else:
    # Call the ligand processing function
    process_ligand_files()